In [1]:
import tensorflow
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
datagen_train = ImageDataGenerator(rescale=1./255)
datagen_test = ImageDataGenerator(rescale=1./255)


In [5]:
train_set = datagen_train.flow_from_directory("../dataset/ttte/train", class_mode='categorical')

Found 0 images belonging to 8 classes.


In [6]:
test_set = datagen_test.flow_from_directory("../dataset/ttte/validation", class_mode='categorical')

Found 23 images belonging to 8 classes.
